<a href="https://colab.research.google.com/github/LanaSina/evolutionary_illusion_generator/blob/master/illusion_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check python version

change python version through menu: Runtime -> Change runtime type

choose python 3 and gpu

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Install libraries

In [ ]:
%pip install -U PyYAML
%pip install neat-python torch
%pip install chainer
# donwgrade numpy
%pip install --upgrade numpy==1.25.2
# downgrade setuptools
%pip install "setuptools <65"

# might become necessary later
# pip install 'cupy-cuda111>=7.7.0,<8.0.0'

Pull code from github

In [ ]:
!rm -r illusion_evolver
!git clone https://github.com/LanaSina/illusion_evolver.git --recurse-submodules #-b cleanup

In [ ]:
# use in case of weird colab encoding error on shell commands
import locale
print(locale.getpreferredencoding())

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

print(locale.getpreferredencoding())


Download prediction model.

black and white model: https://doi.org/10.6084/m9.figshare.13280120

color model: https://figshare.com/articles/Sample_Weight_Model_Front_Psychol_15_March_2018_/11931222

Then drag and drop to the left of this screen, or save in your google drive and change the path below to your own drive.

In [ ]:
# color
!cp '/content/drive/My Drive/AI/prednet_private/models/fpsi_500000_20v.model' '.'

# black and white
#!cp '/content/drive/My Drive/AI/prednet_private/models/black_and_white/300000_wb.model' '.'

Generate illusions. The results will be saved to the "results" folder on the left. Stop the algorithm whenever you are satistified.


In [ ]:
!rm -r "results"
!rm -r "temp"

# argument '-s': Type of illusion. 0: Bands; 1: Circles (default=0)
# g: gradient. 0 = no gradient, 1 = with gradient
# color
%run 'illusion_evolver/generate_illusion.py' -m 'fpsi_500000_20v.model' -s 1 -o './results' -g 1

# black and white
# %run 'illusion_evolver/generate_illusion.py' -m '300000_wb.model' -s 1 -c 1 -o './results' --channels "1,16,32,64" -g 1

# black and white without gradient
# %run 'illusion_evolver/generate_illusion.py' -m '300000_wb.model' -s 1 -c 1 -o './results' --channels "1,16,32,64" -g 0

Dowload results by left-click and "save image as", or download whole folder.



In [ ]:
# download images
!zip -r "results.zip" "results"
from google.colab import files
files.download('results.zip')

Get the visual flow and fitness score of an image (no tiling)

In [ ]:
import os
from PIL import Image

# resize
image_path = 'fw_single.png'
model_name = "300000_wb.model" #for color: 'fpsi_500000_20v.model'
# not used?
channels = [1,16,32,64] #for color: [3,48,96,192]

if not os.path.exists("predictions/"):
    os.makedirs("predictions/")

image = Image.open(image_path).convert("L") # convert('RGB')
#resize with padding rather than squish
maxwidth = 160
maxheight = 120
wpercent = min(maxwidth/image.width, maxheight/image.height)
wsize = int((float(image.width) * float(wpercent)))
hsize = int((float(image.height) * float(wpercent)))
image = image.resize((wsize, hsize), Image.Resampling.LANCZOS)
#image.save("output.png")
# pad with white
# color
# result = Image.new(image.mode, (maxwidth, maxheight), (255, 255, 255))
# bw
result = Image.new(image.mode, (maxwidth, maxheight),255)
left = (int)((maxwidth-wsize)/2)
top = (int)((maxheight-hsize)/2)
result.paste(image, (left, top))
result.save("resized_image.png")

size = [maxwidth,maxheight]

In [ ]:
channels = [1,16,32,64]

In [ ]:
import sys
sys.path.append('illusion_evolver/')

from illusion_evolver import fitness_calculator

vectors = fitness_calculator.get_vectors("resized_image.png", model_name, channels, size[0], size[1])

score = fitness_calculator.calculate_fitness(structure = 1, vectors = vectors,
                                             image_path="resized_image.png",
                                             w = 160, h = 120
                                             )
print(score)